# Project : Custom Cohere with memory

Have you may have seen, there is no memory conversation with our previous LLM. 

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

In this project we will focus on memory.

You will find everything needed here : https://python.langchain.com/docs/modules/memory/ 

### Complete pre-requisite

#### Packages

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain.schema import SystemMessage
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory,
                              ConversationBufferWindowMemory,
                              ConversationKGMemory)
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


# You can add other package here if needed
    

#### Initialize LLM & ConversationChain

A ConversationChain is a type of chain that is specifically designed to handle conversations or dialogues, as opposed to individual inputs.

In [2]:
from langchain.llms import Cohere
from langchain.chains import ConversationChain


# llm = ?

llm = Cohere(temperature=0.75, cohere_api_key=os.environ.get('COHERE_API_KEY'))

# We will use a ConversationChain for our project
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.



### ConversationBufferMemory

The conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form.

In [3]:

# Initialize a ConversationChain with a memory in it

#converstation_buffer_memory = ?

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms import Cohere

conversation_buffer_memory = ConversationBufferMemory()

conversation_with_buffer_memory = ConversationChain(llm=llm,memory=conversation_buffer_memory)

    

#### verification

In [13]:
conversation_with_buffer_memory.invoke("What is the capital of France ?")

conversation_with_buffer_memory.invoke("What was my first question ?")

conversation_with_buffer_memory.invoke("What is the capital of Tunisia ?")

# We should be able to get the whole conversation inside the memory
print(conversation_with_buffer_memory.memory.buffer)

Human: What is the capital of France ?
AI:  The capital of France is Paris. It is located in the north-central part of the country, on the River Seine, and is one of the most populous and well-known cities in the world.

Paris is divided into 20 arrondissements (administrative districts) formed in a spiral pattern around the Louvre Museum and the Opera district. Each arrondissement has its own unique character and attractions, from the vibrant nightlife and cobblestone streets of the Latin Quarter to the upscale shopping avenues and restaurants of the Right Bank. 

Home to iconic landmarks, Paris has a rich historical and cultural heritage. It is renowned for the Eiffel Tower, Notre-Dame Cathedral, the Louvre Museum, the Palace of Versailles, and the Arc de Triomphe, to name just a few. These iconic sights attract millions of tourists every year. 

Beyond these landmarks, Paris has a vibrant atmosphere with something for everyone. It is known for its fashion scene, art museums, haute c

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Allow the AI to remember previous questions
- Simple to implement and re use

##### Cons

- Storing the entire conversation can be a problem because it consumes tokens which can become not free at a fixed number used
- Susceptible to irrelevant information



### ConversationSummaryMemory

The conversation summary memory creates a summary of the conversation over time. 

In [4]:
from langchain.memory import ConversationSummaryMemory


# Initialize ConversationSummaryMemory with your LLM
conversation_summary_memory = ConversationSummaryMemory(llm=llm)

# Initialize ConversationChain with the memory
conversation_with_summary_memory = ConversationChain(
    llm=llm, memory=conversation_summary_memory
)


#### Verification

In [6]:
conversation_with_summary_memory("Gives me the historical chronology of France. I want as much dates and events as possibles.")

{'input': 'Gives me the historical chronology of France. I want as much dates and events as possibles.',
 'history': '',
 'response': ' Sure, here is an expanded timeline of important dates and events in the history of France:\n\n1. 57 BC - Julius Caesar conducts the first invasion of Gaul (modern-day France) and establishes Roman control.\n\n2. 4th Century AD - The capital city of the Roman Empire is moved to Mediolanum (modern-day Milan) causing the decline of the city of Rome. This, along with the arrival of Germanic tribes, signals the fall of the Western Roman Empire. \n\n3. 441 - The Kingdom of the Franks is established by King Clovis I. He unites several Germanic tribes and converts to Christianity, marking an important step in the Catholicization of France.\n\n4. 732 - The Battle of Tours takes place between Charles "The Hammer" Martel\'s Frankish forces and the Umayyad Caliphate. This victory for the Franks marked the end of Muslim expansion into Western Europe.\n\n5. 843 - Th

In [7]:
conversation_with_summary_memory("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

{'input': 'Gives me the historical chronology of Spain. I want as much dates and events as possibles.',
 'history': ' The human asks the AI to provide a detailed chronological timeline of important dates and events in the history of France. The AI responds by providing a list of dates and events, starting from the Roman conquest of Gaul up to the Viking settlements in the late 9th century. ',
 'response': " Ah, the history of Spain is a fascinating one, filled with pivotal events that have shaped the country into what it is today. Let's begin our exploration of key dates and events in Spanish history, starting from the early days of the region.\n\n1. The Iberian Peninsula: Before the arrival of the Romans, the Iberian Peninsula, which today constitutes Spain, was home to various tribes and cultures, including the Iberians, Celts, and Basques. These tribes often clashed with one another, but they were largely unified in their resistance to outside conquests. \n\n2. The Roman Conquest an

In [8]:
conversation_with_summary_memory("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

Retrying langchain_community.llms.cohere.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised CohereAPIError: You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions.


{'input': 'Gives me the historical chronology of Germany. I want as much dates and events as possibles.',
 'history': ' The human asks the AI to provide a detailed chronological timeline of important dates and events in the history of Spain. The AI begins to provide this information, starting from the Roman Conquest of the Iberian Penisula and the Viking Settlements in the late 9th century and then transitioning to the history of Spain specifically. \n\nThe AI outlines the indigenous cultures of the Iberian Penisula, including the Iberians, Celts, and Basques. Then it discusses the Roman Conquest and Influence, including the cultural, infrastructure, and language impacts of Roman rule. This is followed by discussing the Visigoths, who invaded in the 5th century CE. ',
 'response': " Okay, here is a brief timeline of important dates and events in German history:\n\nPre-Unification:\n\n- Late 9th century: Vikings establish the Danelaw in northern Germany and Denmark, which becomes part o

In [9]:
# We should be able to get the whole conversation inside the memory
print(conversation_with_summary_memory.memory.buffer)

 The human asks the AI to provide a detailed chronological timeline of important dates and events in the history of Germany. The AI provides a brief timeline starting from the Roman Conquest of the Iberian Penisula and the Viking Settlements in the late 9th century and then transitioning to specifically German history. They outline the culture, infrastructure, and language impacts of Roman rule and the subsequent Visigoth invasion in the 5th century CE. The AI then provides a summary of important events in German history since pre-unification, including the formation of the Holy Roman Empire, the Hanseatic League, the Age of Exploration, and the founding of the German Empire. 


#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Context awareness, capturing the user intents and the AI responses
- By summarizing the conversation the memory help the AI to focuse on the main topic

##### Cons

- The history part can be too long 
- And it tells only about the last question not the one before



### ConversationSummaryBufferMemory

The conversation summary buffer memory keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions, it compiles them into a summary and uses both.

In [10]:

# Initialize a ConversationChain with a memory in it

# converstation_summary_buffer_memory = ConversationChain(....)

from langchain.memory import ConversationSummaryBufferMemory

# Initialize ConversationSummaryBufferMemory
conversation_summary_buffer_memory = ConversationSummaryBufferMemory(llm=llm)

# Initialize ConversationChain with the memory
conversation_with_summary_buffer_memory = ConversationChain(
    llm=llm, memory=conversation_summary_buffer_memory
)


    

#### Verification

In [11]:
conversation_with_summary_buffer_memory("Gives me the historical chronology of France. I want as much dates and events as possibles.")

{'input': 'Gives me the historical chronology of France. I want as much dates and events as possibles.',
 'history': '',
 'response': ' Sure, here is an expanded version of the chronological timeline of important dates and events in the history of France, presented in a list format for clarity:\n\n1. **c. 4500 BCE** - Neanderthals and modern humans coexist in present-day France, with Neanderthals likely retreating or dying out around this time.\n\n2. **c. 4000 BCE** - The onset of the Neolithic Revolution sees the introduction of agriculture and the creation of the first permanent settlements in the country.\n\n3. **1200 BCE** - Celtic tribes, notably the Gauls, arrive and settle in various regions of what will become France. They establish a Celtic culture that influences language, religion, and societal structures.\n\n4. **c. 550 BCE** - The Celtic Hallstatt culture flourishes in Gaul, marking technological advancements, expanded trade networks, and stylized metalwork.\n\n5. **c. 390

In [12]:
conversation_with_summary_buffer_memory("Gives me the historical chronology of Spain. I want as much dates and events as possibles.")

{'input': 'Gives me the historical chronology of Spain. I want as much dates and events as possibles.',
 'history': 'Human: Gives me the historical chronology of France. I want as much dates and events as possibles.\nAI:  Sure, here is an expanded version of the chronological timeline of important dates and events in the history of France, presented in a list format for clarity:\n\n1. **c. 4500 BCE** - Neanderthals and modern humans coexist in present-day France, with Neanderthals likely retreating or dying out around this time.\n\n2. **c. 4000 BCE** - The onset of the Neolithic Revolution sees the introduction of agriculture and the creation of the first permanent settlements in the country.\n\n3. **1200 BCE** - Celtic tribes, notably the Gauls, arrive and settle in various regions of what will become France. They establish a Celtic culture that influences language, religion, and societal structures.\n\n4. **c. 550 BCE** - The Celtic Hallstatt culture flourishes in Gaul, marking techn

In [13]:
conversation_with_summary_buffer_memory("Gives me the historical chronology of Germany. I want as much dates and events as possibles.")

{'input': 'Gives me the historical chronology of Germany. I want as much dates and events as possibles.',
 'history': 'Human: Gives me the historical chronology of France. I want as much dates and events as possibles.\nAI:  Sure, here is an expanded version of the chronological timeline of important dates and events in the history of France, presented in a list format for clarity:\n\n1. **c. 4500 BCE** - Neanderthals and modern humans coexist in present-day France, with Neanderthals likely retreating or dying out around this time.\n\n2. **c. 4000 BCE** - The onset of the Neolithic Revolution sees the introduction of agriculture and the creation of the first permanent settlements in the country.\n\n3. **1200 BCE** - Celtic tribes, notably the Gauls, arrive and settle in various regions of what will become France. They establish a Celtic culture that influences language, religion, and societal structures.\n\n4. **c. 550 BCE** - The Celtic Hallstatt culture flourishes in Gaul, marking tec

In [14]:
# We should be able to get the whole conversation inside the memory
print(conversation_with_summary_buffer_memory.memory.buffer)

[HumanMessage(content='Gives me the historical chronology of France. I want as much dates and events as possibles.'), AIMessage(content=' Sure, here is an expanded version of the chronological timeline of important dates and events in the history of France, presented in a list format for clarity:\n\n1. **c. 4500 BCE** - Neanderthals and modern humans coexist in present-day France, with Neanderthals likely retreating or dying out around this time.\n\n2. **c. 4000 BCE** - The onset of the Neolithic Revolution sees the introduction of agriculture and the creation of the first permanent settlements in the country.\n\n3. **1200 BCE** - Celtic tribes, notably the Gauls, arrive and settle in various regions of what will become France. They establish a Celtic culture that influences language, religion, and societal structures.\n\n4. **c. 550 BCE** - The Celtic Hallstatt culture flourishes in Gaul, marking technological advancements, expanded trade networks, and stylized metalwork.\n\n5. **c. 3

#### From your observations - Gives the pros and cons of this memory 

##### Pros

- Stores the whole conversation with human questions and AI answers, can be benefic if we want to ask a new question about a detail given by in the previous answers
- Have advantages of of ConversationSummaryMemory and ConversationBufferMemory

##### Cons

- It seems that the summary of the previous question in the history part don't work well, actually it returns the whole question given by the human and the wole AI answer, in the second part (ConversationSummaryMemory) we find a summary of the previous question in the history part
- Redundancy of information